# data 불러오기
- import version 설정(터미널에서 지정)
    * pip install scikit-learn==1.1
    * pip install imbalanced-learn==0.9



## data 확인
데이터 출처: Kaggle
https://www.kaggle.com/code/joshuaswords/awesome-hr-data-visualization-prediction

# Modeling

In [116]:
from sklearn.model_selection import train_test_split

X = aug_train.dropna().drop(columns=['target']).values
y = aug_train.dropna()['target'].values

# train, val, test 셋 분리하는 코드
# 데이터셋 분할: Train/Validation + Test
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.2, random_state=80)
# 데이터셋 분할: Train + Validation
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.2, random_state=80)

In [117]:
# 열 이름 설정
feature_columns = aug_train.drop(columns=['target']).columns
target_column = ['target']

# numpy 배열을 DataFrame으로 변환
X_train_df = pd.DataFrame(X_train_val, columns=feature_columns)
X_val_df = pd.DataFrame(X_val, columns=feature_columns)
X_test_df = pd.DataFrame(X_test, columns=feature_columns)
y_train_df = pd.DataFrame(y_train_val, columns=target_column)
y_val_df = pd.DataFrame(y_val, columns=target_column)
y_test_df = pd.DataFrame(y_test, columns=target_column)

# Ensure 'train_df' and 'test_df' are created correctly by concatenating X and y
# train_df : for visuals
train_df = pd.concat([X_train_df, y_train_df], axis=1)
test_df = pd.concat([X_val_df, y_val_df], axis=1)
new_data_df = pd.concat([X_test_df, y_test_df], axis=1)

# 출력 확인 
print(train_df.info())
print(test_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15326 entries, 0 to 15325
Data columns (total 32 columns):
 #   Column                                      Non-Null Count  Dtype  
---  ------                                      --------------  -----  
 0   city_development_index                      15326 non-null  float64
 1   education_level                             15326 non-null  float64
 2   experience                                  15326 non-null  float64
 3   company_size                                15326 non-null  float64
 4   last_new_job                                15326 non-null  float64
 5   training_hours                              15326 non-null  float64
 6   experience_under_1                          15326 non-null  float64
 7   experience_over_20                          15326 non-null  float64
 8   experience_Unknown                          15326 non-null  float64
 9   gender_Male                                 15326 non-null  float64
 10  gender_Oth

In [118]:
X_train, X_test, y_train, y_test = X_train, X_val, y_train, y_val

In [119]:
import pandas as pd
import numpy as np
import seaborn as sns

from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE

from sklearn.metrics import accuracy_score, recall_score, precision_score, roc_auc_score, f1_score, confusion_matrix

from sklearn.svm import LinearSVC, SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier

In [137]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_scaled, y_train)

In [139]:
# Assume 'target' is the target column and the rest are features
X_new_test = new_data_df.drop(columns=['target'])
y_new_test = new_data_df['target']

# Convert to numpy arrays for consistency
X_new_test_array = X_new_test.values

# 전처리 (Scaling) 적용
X_new_test_scaled = scaler.fit_transform(X_new_test_array)


def test_data(model, model_t):
    prediction = model.predict(X_new_test_scaled)
    prediction_t = model_t.predict(X_new_test_scaled)

    cm_new = confusion_matrix(y_new_test, prediction)
    df_new = pd.DataFrame(
        data=[
            f1_score(y_new_test, prediction),
            recall_score(y_new_test, prediction),
            precision_score(y_new_test, prediction),
            accuracy_score(y_new_test, prediction),
            roc_auc_score(y_new_test, prediction)
        ],
        columns=[f'{model} Score'],
        index=["F1 Score", "Recall", "Precision", "Accuracy", "ROC AUC Score"]
    )

    cm_new_t = confusion_matrix(y_new_test, prediction_t)
    df_new_t = pd.DataFrame(
        data=[
            f1_score(y_new_test, prediction_t),
            recall_score(y_new_test, prediction_t),
            precision_score(y_new_test, prediction_t),
            accuracy_score(y_new_test, prediction_t),
            roc_auc_score(y_new_test, prediction_t)
        ],
        columns=[f'Tuned {model_t} Score'],
        index=["F1 Score", "Recall", "Precision", "Accuracy", "ROC AUC Score"]
    )

    # 결과 출력
    print("Model result")
    print(cm_new)
    print(df_new)

    # 결과 출력
    print("Tuned Model result")
    print(cm_new_t)
    print(df_new_t)


## SVM

### Poly

In [140]:
SVC_poly = SVC(kernel='poly')
SVC_poly.fit(X_train_resampled, y_train_resampled)

# 예측 및 평가
SVC_poly_prediction = SVC_poly.predict(X_test_scaled)
cm_SVC_poly = confusion_matrix(y_test, SVC_poly_prediction)
df_SVC_poly = pd.DataFrame(data=[f1_score(y_test, SVC_poly_prediction), recall_score(y_test, SVC_poly_prediction),
                   precision_score(y_test, SVC_poly_prediction), accuracy_score(y_test, SVC_poly_prediction), 
                   roc_auc_score(y_test, SVC_poly_prediction)],
             columns=['SVC Poly Kernel Score'],
             index=["F1 Score", "Recall", "Precision", "Accuracy", "ROC AUC Score"])
df_SVC_poly

,SVC Poly Kernel Score
F1 Score,0.587034
Recall,0.740979
Precision,0.486052
Accuracy,0.736138
ROC AUC Score,0.737739


In [141]:
params_grid = {
    'C': [10 ** k for k in range(-3, 3)],    
    # 'gamma': ['scale', 'auto', 0.01, 0.1, 1],
    # 'degree': [2, 3, 4, 5] # 3
}

# Initialize the GridSearchCV
clf = GridSearchCV(SVC_poly, params_grid, cv=3, scoring='f1')
clf.fit(X_train_resampled, y_train_resampled)

# Output the best parameters
print("Best Parameters: ", clf.best_params_)

# Predictions and evaluation
SVC_poly_t_prediction = clf.predict(X_test_scaled)
cm_SVC_poly_t = confusion_matrix(y_test, SVC_poly_t_prediction)
df_SVC_poly_t = pd.DataFrame(data=[f1_score(y_test, SVC_poly_t_prediction), recall_score(y_test, SVC_poly_t_prediction),
                   precision_score(y_test, SVC_poly_t_prediction), accuracy_score(y_test, SVC_poly_t_prediction), 
                   roc_auc_score(y_test, SVC_poly_t_prediction)],
             columns=['Tuned SVC Poly Kernel Score'],
             index=["F1 Score", "Recall", "Precision", "Accuracy", "ROC AUC Score"])
df_SVC_poly_t

Best Parameters:  {'C': 100}


,Tuned SVC Poly Kernel Score
F1 Score,0.575615
Recall,0.708763
Precision,0.484581
Accuracy,0.735486
ROC AUC Score,0.726652


In [142]:
test_data(SVC_poly, clf.best_estimator_)

Model result
[[2103  756]
 [ 252  721]]
               SVC(kernel='poly') Score
F1 Score                       0.588571
Recall                         0.741007
Precision                      0.488152
Accuracy                       0.736952
ROC AUC Score                  0.738290
Tuned Model result
[[2129  730]
 [ 295  678]]
               Tuned SVC(C=100, kernel='poly') Score
F1 Score                                    0.569509
Recall                                      0.696814
Precision                                   0.481534
Accuracy                                    0.732516
ROC AUC Score                               0.720740


### RBF

In [143]:
SVC_rbf = SVC(kernel='rbf')
SVC_rbf.fit(X_train_resampled, y_train_resampled)

# 예측 및 평가
SVC_RBF_prediction = SVC_rbf.predict(X_test_scaled)
cm_SVC_RBF = confusion_matrix(y_test, SVC_RBF_prediction)
df_SVC_RBF = pd.DataFrame(data=[f1_score(y_test, SVC_RBF_prediction), recall_score(y_test, SVC_RBF_prediction),
                   precision_score(y_test, SVC_RBF_prediction), accuracy_score(y_test, SVC_RBF_prediction), 
                   roc_auc_score(y_test, SVC_RBF_prediction)],
             columns=['SVC RBF Kernel Score'],
             index=["F1 Score", "Recall", "Precision", "Accuracy", "ROC AUC Score"])
df_SVC_RBF

,SVC RBF Kernel Score
F1 Score,0.587571
Recall,0.737113
Precision,0.488471
Accuracy,0.738095
ROC AUC Score,0.737771


In [144]:
params_grid = {
    'C': [10 ** k for k in range(-4, 4)],    
    'gamma': ['scale', 'auto', 0.01, 0.1, 1]
}

# Initialize the GridSearchCV
SVC_rbf_t = GridSearchCV(SVC_rbf, params_grid, cv=5, scoring='f1')
SVC_rbf_t.fit(X_train_resampled, y_train_resampled)

# Output the best parameters
print("Best Parameters: ", SVC_rbf_t.best_params_)

# Predictions and evaluation
SVC_RBF_t_prediction = SVC_rbf_t.predict(X_test_scaled)
cm_SVC_RBF_t = confusion_matrix(y_test, SVC_RBF_t_prediction)
df_SVC_RBF_t = pd.DataFrame(data=[f1_score(y_test, SVC_RBF_t_prediction), recall_score(y_test, SVC_RBF_t_prediction),
                   precision_score(y_test, SVC_RBF_t_prediction), accuracy_score(y_test, SVC_RBF_t_prediction), 
                   roc_auc_score(y_test, SVC_RBF_t_prediction)],
             columns=['Tuned SVC RBF Kernel Score'],
             index=["F1 Score", "Recall", "Precision", "Accuracy", "ROC AUC Score"])
df_SVC_RBF_t

Best Parameters:  {'C': 10, 'gamma': 1}


,Tuned SVC RBF Kernel Score
F1 Score,0.431880
Recall,0.408505
Precision,0.458092
Accuracy,0.727984
ROC AUC Score,0.622375


In [146]:
test_data(SVC_rbf, SVC_rbf_t.best_estimator_)

Model result
[[2121  738]
 [ 241  732]]
               SVC() Score
F1 Score          0.599263
Recall            0.752312
Precision         0.497959
Accuracy          0.744520
ROC AUC Score     0.747090
Tuned Model result
[[2408  451]
 [ 578  395]]
               Tuned SVC(C=10, gamma=1) Score
F1 Score                             0.434305
Recall                               0.405961
Precision                            0.466903
Accuracy                             0.731472
ROC AUC Score                        0.624107
